# Load Config

In [14]:
import json
import os

In [ ]:
def load_app_config(config_file):
    config_path = os.path.join(config_file)
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        return json.load(f)

cfg = load_app_config("config_mymlops_dev.json")

# Resource Group

In [17]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

In [18]:
rg_config = cfg.get("resource_group", {})
subscription_id = cfg.get("subscription_id", {})
rg_name = rg_config.get("name")
location = rg_config.get("location", "eastus")

In [19]:
def get_resource_client(subscription_id):
    """Authenticate and return the ResourceManagementClient"""
    credential = DefaultAzureCredential()
    return ResourceManagementClient(credential, subscription_id)

In [20]:
resource_client = get_resource_client(subscription_id)

In [21]:
def validate_resource_group(resource_client, rg_name):
    """Check if a resource group exists"""
    print(f"🔍 Validating resource group '{rg_name}'...")
    exists = resource_client.resource_groups.check_existence(rg_name)
    if exists:
        print(f"✅ Resource group '{rg_name}' exists.")
    else:
        print(f"❌ Resource group '{rg_name}' does not exist.")
    return exists

### Validating if resource group is available if not create one

In [22]:
if validate_resource_group(resource_client, rg_name):
    print(f"ℹ️ Resource group '{rg_name}' already exists.")
else:
    print(f"🚀 Creating resource group '{rg_name}' in '{location}'...")
    rg_result = resource_client.resource_groups.create_or_update(
        rg_name, {"location": location}
    )
    print(f"✅ Resource group '{rg_result.name}' created successfully.")

🔍 Validating resource group 'rg-mymlops'...
❌ Resource group 'rg-mymlops' does not exist.
🚀 Creating resource group 'rg-mymlops' in 'eastus'...
✅ Resource group 'rg-mymlops' created successfully.


# Virtual machine

In [23]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import (
    HardwareProfile, OSProfile, LinuxConfiguration,
    SshConfiguration, SshPublicKey, NetworkInterfaceReference,
    StorageProfile, ImageReference, OSDisk, DiskCreateOptionTypes
)
from azure.mgmt.network.models import (
    NetworkSecurityGroup, SecurityRule, PublicIPAddress,
    NetworkInterface, Subnet, VirtualNetwork, IPConfiguration
)

In [24]:
subscription_id = cfg["subscription_id"]
resource_group = cfg["resource_group"]["name"]
location = cfg["resource_group"]["location"]
vm_name = cfg["vm"]["name"]
vm_size = cfg["vm"]["size"]
admin_username = cfg["vm"]["admin_username"]
ssh_key_path = os.path.expanduser(cfg["vm"]["ssh_public_key_path"])
virtual_network = cfg["vm"]["virtual_network"]
sub_network = cfg["vm"]["sub_network"]
network_security_grp = cfg["vm"]["network_security_grp"]
public_ip = cfg["vm"]["public_ip"]
network_interface = cfg["vm"]["network_interface"]

In [25]:
print(f"🚀 Starting VM creation: {vm_name} in {location}")

credential = DefaultAzureCredential()
resource_client = ResourceManagementClient(credential, subscription_id)
network_client = NetworkManagementClient(credential, subscription_id)
compute_client = ComputeManagementClient(credential, subscription_id)

🚀 Starting VM creation: vm-mlops in eastus


In [26]:
print(f"🔹 Ensuring resource group '{resource_group}' exists...")
resource_client.resource_groups.create_or_update(
    resource_group, {"location": location})

🔹 Ensuring resource group 'rg-mymlops' exists...


In [ ]:
# Virtual Network
print("🔹 Creating Virtual Network...")
vnet = network_client.virtual_networks.begin_create_or_update(
    resource_group, virtual_network,
    {
        "location": location, 
        "address_space": {"address_prefixes": ["10.0.0.0/16"]}
    },
).result()
print(f"✅ Virtual Network '{vnet.name}' created successfully.")

🔹 Creating Virtual Network...


HttpResponseError: (ResourceNotFound) The Resource 'Microsoft.Network/virtualNetworks/vnet-mlops' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Network/virtualNetworks/vnet-mlops' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [29]:
# Subnet
print("🔹 Creating Subnet...")
subnet = network_client.subnets.begin_create_or_update(
    resource_group, virtual_network, 
    sub_network, {"address_prefix": "10.0.0.0/24"},
).result()
print(f"✅ Subnet '{subnet.name}' created successfully.")

🔹 Creating Subnet...
✅ Subnet 'subnet-mlops' created successfully.


In [30]:
# NSG (SSH only)
print("🔹 Creating NSG (allow SSH)...")
nsg = network_client.network_security_groups.begin_create_or_update(
    resource_group, network_security_grp,
    {
        "location": location,
        "security_rules": [
            SecurityRule(
                name="AllowSSH", protocol="Tcp", direction="Inbound", 
                access="Allow", priority=1000, source_address_prefix="*", 
                destination_address_prefix="*", source_port_range="*", 
                destination_port_range="22",
            )
        ],
    },
).result()
print(f"✅ NSG '{nsg.name}' created successfully.")

🔹 Creating NSG (allow SSH)...
✅ NSG 'nsg-mlops' created successfully.


In [ ]:
# Public IP
print("🔹 Creating Public IP...")
public_ip = network_client.public_ip_addresses.begin_create_or_update(
    resource_group, public_ip, 
    {"location": location, "public_ip_allocation_method":"Dynamic"},
    ).result()
print(f"✅ NSG '{public_ip.name}' created successfully.")
# print(f"✅ NSG '{public_ip.ip_address}' created successfully.")

🔹 Creating Public IP...
✅ NSG 'pip-mlops' created successfully.


In [35]:
# Network Interface
print("🔹 Creating Network Interface...")
nic = network_client.network_interfaces.begin_create_or_update(
    resource_group, network_interface,
    {
        "location": location,
        "ip_configurations": [
            IPConfiguration(
                name="ipconfig1",
                subnet=Subnet(id=subnet.id),
                public_ip_address=PublicIPAddress(id=public_ip.id),
                network_security_group=NetworkSecurityGroup(id=nsg.id),
            )
        ],
    },
).result()
print(f"✅ NSG '{nic.name}' created successfully.")

network_security_group is not a known attribute of class <class 'azure.mgmt.network.models._models_py3.IPConfiguration'> and will be ignored


🔹 Creating Network Interface...
✅ NSG 'nic-mlops' created successfully.


In [40]:
# VM Configuration
print("🔹 Defining VM configuration...")
with open(ssh_key_path, "r") as f:
    ssh_key_data = f.read().strip()

vm_parameters = {
    "location": location,
    "hardware_profile": HardwareProfile(vm_size=vm_size),
    "storage_profile": StorageProfile(
        image_reference=ImageReference(
            publisher="Canonical",
            offer="UbuntuServer",
            sku="18.04-LTS",
            version="latest",
        ),
        os_disk=OSDisk(
            create_option=DiskCreateOptionTypes.from_image,
            managed_disk={"storage_account_type": "StandardSSD_LRS"},
        ),
    ),
    "os_profile": OSProfile(
        computer_name=vm_name,
        admin_username=admin_username,
        linux_configuration=LinuxConfiguration(
            disable_password_authentication=True,
            ssh=SshConfiguration(
                public_keys=[
                    SshPublicKey(
                        path=f"/home/{admin_username}/.ssh/authorized_keys",
                        key_data=ssh_key_data,
                    )
                ]
            ),
        ),
    ),
    "network_profile": {
        "network_interfaces": [NetworkInterfaceReference(id=nic.id)]},
}

🔹 Defining VM configuration...


In [41]:
# Create the VM
print(f"🚀 Creating VM '{vm_name}' ... this may take a few minutes.")
creation = compute_client.virtual_machines.begin_create_or_update(
    resource_group, vm_name, vm_parameters
)
vm_result = creation.result()

🚀 Creating VM 'vm-mlops' ... this may take a few minutes.


'vm-mlops'